In [31]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm
from collections import deque, namedtuple

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)
n_cols = len(input_text[0])
for ii in range(1, n_rows):
    assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 130 rows and 130 cols


In [29]:
mat_visited = np.zeros((n_rows, n_cols))
for i_r, r in enumerate(input_text):
    if '^' in r:
        start_r = i_r
        for i_c, c in enumerate(r):
            if c == '^':
                start_c = i_c
                break 
        break 

start_r, start_c
assert input_text[start_r][start_c] == '^'

mat_visited[start_r, start_c] = 1
dirs = deque([(-1, 0), (0, 1), (1, 0), (0, -1)])
curr_dir = dirs.popleft()
seen = set((start_r, start_c, curr_dir))
keep_searching = True 
it = 0
max_it = 200000

while keep_searching and it < max_it:
    it += 1
    next_r = i_r + curr_dir[0]
    next_c = i_c + curr_dir[1]

    if next_r < 0 or next_c < 0 or next_r >= n_rows or next_c >= n_cols:  #out of bounds
        keep_searching = False
        break 
    elif (next_r, next_c, curr_dir) in seen:  # in a loop
        keep_searching = False
        break 
    elif input_text[next_r][next_c] == '#':  # turn right
        dirs.append(curr_dir)
        curr_dir = dirs.popleft()
    else:  # step ahead
        mat_visited[next_r, next_c] = 1
        seen.add((next_r, next_c, curr_dir))
        i_r, i_c = next_r, next_c 

print(mat_visited.sum())

5177.0


## part 2

In [39]:
## part 2 brute force possible? Put everything in function and iterate through new obstacles

mat_visited = np.zeros((n_rows, n_cols))
for i_r, r in enumerate(input_text):
    if '^' in r:
        start_r = i_r
        for i_c, c in enumerate(r):
            if c == '^':
                start_c = i_c
                break 
        break 

start_r, start_c
assert input_text[start_r][start_c] == '^'

mat_visited[start_r, start_c] = 1

def run(map):
    i_r, i_c = start_r, start_c
    is_loop = False
    dirs = deque([(-1, 0), (0, 1), (1, 0), (0, -1)])
    curr_dir = dirs.popleft()
    seen = set((start_r, start_c, curr_dir))
    keep_searching = True 
    it = 0
    max_it = 200000

    while keep_searching and it < max_it:
        it += 1
        next_r = i_r + curr_dir[0]
        next_c = i_c + curr_dir[1]

        if next_r < 0 or next_c < 0 or next_r >= n_rows or next_c  >= n_cols:  # end
            keep_searching = False
            break 
        elif (next_r, next_c, curr_dir) in seen:
            keep_searching = False
            is_loop = True
            return 1
            break 
        elif map[next_r][next_c] == '#':  # turn right
            dirs.append(curr_dir)
            curr_dir = dirs.popleft()
            
        else:
            # mat_visited[next_r, next_c] = 1
            seen.add((next_r, next_c, curr_dir))
            i_r, i_c = next_r, next_c 

    return 0 

run(input_text)

0

In [44]:
%timeit run(input_text)
## yes bit lazy but works

1.57 ms ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [42]:
total = 0
for iir in range(n_rows):
    for iic in range(n_cols):
        if input_text[iir][iic] == '.':  # temporarily add wall, run, and remove wall
            tmp = list(input_text[iir])
            tmp[iic] = '#'
            input_text[iir] = ''.join(tmp)
            total += run(map=input_text)
            tmp[iic] = '.'
            input_text[iir] = ''.join(tmp)

print(total)

1686
